<a href="https://colab.research.google.com/github/delaiglesialab/DigitalRhythmsProject/blob/main/es/espa%C3%B1ol_Android_Timestamps_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Obtener Las marcas de tiempo de Android**

---

¡Gracias por ayudarnos a investigar los ritmos de trabajo bajo aislamiento social!

No se preocupe si no está familiarizado con estos notebooks, siga las instrucciones en cada "bloque" llamado celda y presione el botón de *play* a la izquierda para cada una de ellas de manera secuencial (a menos que se indique lo contrario). ¡Con unos pocos clics estará completo el proceso!

Si ve algún error o no ejecuto las celdas en orden, ¡Simplemente vuelva a cargar la página y comience de nuevo!

---

Cargará el archivo de datos HTML que descargó de su cuenta de Google a través de Google Takeout. Sólo se almacena temporalmente y lo guiaremos para eliminarlo al final de su sesión.

Nuestro código hará un procesamiento simple por usted, y luego obtendrá un archivo CSV que contiene las marcas de tiempo de uso de su teléfono que pueden ser utilizadas para el análisis.

Hay una celda de "filtro de privacidad" opcional, sólo ejecútela si también desea borrar el nombre de las aplicaciones. Los datos no se pueden rastrear hasta usted y si suficientes personas proporcionan datos con nombres de aplicaciones, podremos hacer análisis más interesantes con ellos, así que considere compartir los datos tal como están.

---

¡Buena suerte y no dudes en comunicarte con Leandro a lcasira@uw.edu si tienes algún problema!



---









In [ ]:
#@title Configuración
#@markdown comience ejecutando esta celda con el botón de la izquierda. ¡Puede tomar un poco de tiempo ya que carga todo para ti!
from google.colab import files
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.parser import parse

In [ ]:
#@title Subir archivo de datos
#@markdown Al ejecutar esta celda, se le pedirá que cargue su archivo HTML que contiene marcas de tiempo de Android.

#@markdown Nota importante: su archivo HTML DEBE llamarse "My Activity.html" EXACTAMENTE, o nuestro código no funcionará. Cambie el nombre de su archivo si es necesario.
file = files.upload()

os.rename(r'My Activity.html',r'actdata.txt')
with open('actdata.txt','r') as file:
  data = file.readlines()
print("Data loaded from file.")
!rm *.* #removes file, bash shell command
print("File deleted from Colab, verify in files on left panel.")

In [ ]:
#@title Crear marco de datos
#@markdown La ejecución de esta celda extrae los datos y los coloca en un marco de datos, una vez ejecutado, le mostrará las primeras y las últimas entradas de sus datos.

actdat = data[32]

preapp = [app.end(0) for app in re.finditer('<p class="mdl-typography--title">', actdat)] 
postapp = [app.start(0) for app in re.finditer('<br></p></div><div class="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1">', actdat)]
posttime = [time.start(0) for time in re.finditer('</div><div class="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1 mdl-typography--text-right">', actdat)]

appname = []
datetime = []
timezone = []

for i in range(len(posttime)):

  appname += [actdat[preapp[i]:postapp[i]]]

  stamp = actdat[posttime[i]-30:posttime[i]]
  idx = [app.end(0) for app in re.finditer('>', stamp)]
  if len(idx) > 0:
    stamp = stamp[idx[-1]:]

  datetime += [parse(stamp[:-4])]
  timezone += [stamp[-3:]]

dataframe = pd.DataFrame(list(zip(appname,datetime,timezone)),columns = ['App','Timestamp','TimeZone'])
print(dataframe.head())

In [ ]:
#@title Filtro de privacidad
#@markdown ** ¡Considere NO usar esta celda! **

#@markdown Esta celda elimina los nombres de las aplicaciones de los datos. Se cambia el nombre de todas las aplicaciones a "aplicación", a excepción de las aplicaciones de reloj / alarma, que se denominan "reloj".

dataframe.loc[dataframe['App'].str.contains('clock', case=False), 'App'] = 'clock'
dataframe.loc[dataframe['App'] != "clock", "App"] = "app"

print("appnames have been removed, replaced with "+str(dataframe.App.unique())+"\n\n")
print(dataframe.head())

In [ ]:
#@title Descargar el marco de datos como csv
#@markdown Esta celda le permite descargar el CSV. Tenga en cuenta que se llama "AllData.csv" y se le pedirá que le cambie el nombre en el siguiente paso antes de enviarlo.
dataframe.to_csv("AllData.csv")
files.download("AllData.csv")

In [ ]:
#@title Pizarra limpia
#@markdown Esto elimina todos los archivos de colab cuando se ejecuta, incluido el archivo HTML y el archivo CSV que descargó sin dejar rastro.

!rm *.*
print("¡Listo y desempolvado!")

---

¡Listo!

¡Continúe con el paso 2 de la página principal!

---